# Monitoring
Пример отчета по расчету статистик в рамках проекта "Мониторинг рекламы в интернете".

## Описание задачи

Посчитаем статистики по рекламе маркетплейсов с детализацией по рекламодателям и брендам.

Общие параметры:
- Период: 1 - 31 января 2023
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: Web Desktop
- Тип рекламы: Видео

Статистики:
- ots
- sov
- reach
- reachPer (Reach %)
- frequency
- days (количество дней размещения)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Задание

### Получим ID категории товаров и услуг 4-ого уровня "Маркетплейсы"
Для построения отчета необходимо получить идентификатор категории товаров и услуг 4-ого уровня "Маркетплейсы". 

Для этого воспользуемся методами поиска в справочнике атрибутов описания креатива. Работа со справочником описана в ноутбуке [catalogs_monitoring](catalogs_monitoring.ipynb). 

Получим идентификатор категории 4-ого уровня **Маркетплейсы**

In [ ]:
cats.get_product_category_l4(product_category_l4="Маркетплейсы")

Таким образом, необходимый идентификатор следующий Id = 4977

### Расчет статистик по рекламе на маркетплейсах с разбивкой по рекламодателям и брендам

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Marketplace'

# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2023-01-01', '2023-01-31')]

# Задаем фильтр по типам пользования интернетом, в нашем случае Web Desktop (1)
usetype_filter = [1]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по атрибутам размещения рекламы, в нашем случае он не требуется
event_description_filter = None

# Задаем фильтр по атрибутам описания креатива, в нашем случае это ID категории товаров и услуг 4-ого уровня "Маркетплейсы"
ad_description_filter = 'productCategoryL4Id = 4977'

# Задаем фильтр по медиа объекту, на котором размещалась рекламы, в нашем случае он не требуется
mart_filter = None

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["advertiserName", "productBrandName"]

# Указываем список статистик для расчета
statistics = ["ots", "sov", "reach", "reachPer", "frequency", "days"]

# Если выбрана статистика reachN, необходимо указать для нее шкалу 
# scales = {'reachN':[(2, 4), (5,10)]}

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task_monitoring('monitoring', project_name, date_filter=date_filter, 
                             usetype_filter=usetype_filter, geo_filter=geo_filter,
                             demo_filter=demo_filter, mart_filter=mart_filter, 
                             ad_description_filter=ad_description_filter,
                             event_description_filter=event_description_filter, 
                             slices=slices, statistics=statistics, scales=None)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_task(task_json))

# Получаем результат
df_monitoring = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_monitoring

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('monitoring'))
df_info = mtask.get_report_info()
df_monitoring.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()